In [1]:
import torch
import torch.onnx

In [2]:
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 73.4 MB/s eta 0:00:00


In [3]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.4 MB/s eta 0:00:00


# Convert Pretrained model to ONNX

In [4]:
import timm

# Create the EfficientNetV2 model architecture
pretrained_model = timm.create_model('tf_efficientnetv2_b2', pretrained=True)


# Load the checkpoint
filepath = 'drive/MyDrive/cats_vs_dogs_task/Efficientnetv2_b2/efficientnetv2_b2.h5'
checkpoint = torch.load(filepath, map_location='cpu')


# Load the state_dict into the model's layers
pretrained_model.load_state_dict(checkpoint)
pretrained_model.eval()

# Print the model architecture
print(pretrained_model)

EfficientNet(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): ConvBnAct(
        (conv): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
      (1): ConvBnAct(
        (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
    )
    (1): Sequential(
   

In [5]:
save_path_pre = 'drive/MyDrive/cats_vs_dogs_task/ONNX/Efficientnetv2b2Pre.onnx'
dummy_input = dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(pretrained_model, dummy_input, save_path_pre)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [7]:
import onnx

# Load the ONNX model
model = onnx.load(save_path_pre)

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch_jit (
  %x.1[FLOAT, 1x3x224x224]
) initializers (
  %blocks.3.0.se.conv_reduce.weight[FLOAT, 14x224x1x1]
  %blocks.3.0.se.conv_reduce.bias[FLOAT, 14]
  %blocks.3.0.se.conv_expand.weight[FLOAT, 224x14x1x1]
  %blocks.3.0.se.conv_expand.bias[FLOAT, 224]
  %blocks.3.1.se.conv_reduce.weight[FLOAT, 26x416x1x1]
  %blocks.3.1.se.conv_reduce.bias[FLOAT, 26]
  %blocks.3.1.se.conv_expand.weight[FLOAT, 416x26x1x1]
  %blocks.3.1.se.conv_expand.bias[FLOAT, 416]
  %blocks.3.2.se.conv_reduce.weight[FLOAT, 26x416x1x1]
  %blocks.3.2.se.conv_reduce.bias[FLOAT, 26]
  %blocks.3.2.se.conv_expand.weight[FLOAT, 416x26x1x1]
  %blocks.3.2.se.conv_expand.bias[FLOAT, 416]
  %blocks.3.3.se.conv_reduce.weight[FLOAT, 26x416x1x1]
  %blocks.3.3.se.conv_reduce.bias[FLOAT, 26]
  %blocks.3.3.se.conv_expand.weight[FLOAT, 416x26x1x1]
  %blocks.3.3.se.conv_expand.bias[FLOAT, 416]
  %blocks.4.0.se.conv_reduce.weight[FLOAT, 26x624x1x1]
  %blocks.4.0.se.conv_reduce.bias[FLOAT, 26]
  %blocks.4.0.se.conv_expand.weigh

# Convert scratch model to ONNX

In [8]:
# Create the EfficientNetV2 model architecture
scratch_model = timm.create_model('tf_efficientnetv2_b2', pretrained=False)


# Load the checkpoint
filepath = 'drive/MyDrive/cats_vs_dogs_task/Efficientnetv2_b2_NP/Efficientnetv2_b2_NP.h5'
checkpoint = torch.load(filepath, map_location='cpu')


# Load the state_dict into the model's layers
scratch_model.load_state_dict(checkpoint)
scratch_model.eval()

# Print the model architecture
print(scratch_model)

EfficientNet(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): ConvBnAct(
        (conv): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
      (1): ConvBnAct(
        (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
    )
    (1): Sequential(
   

In [9]:
save_path_scr = 'drive/MyDrive/cats_vs_dogs_task/ONNX/Efficientnetv2b2Scr.onnx'
dummy_input = dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(pretrained_model, dummy_input, save_path_scr)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [10]:
import onnx

# Load the ONNX model
model = onnx.load(save_path_scr)

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch_jit (
  %x.1[FLOAT, 1x3x224x224]
) initializers (
  %blocks.3.0.se.conv_reduce.weight[FLOAT, 14x224x1x1]
  %blocks.3.0.se.conv_reduce.bias[FLOAT, 14]
  %blocks.3.0.se.conv_expand.weight[FLOAT, 224x14x1x1]
  %blocks.3.0.se.conv_expand.bias[FLOAT, 224]
  %blocks.3.1.se.conv_reduce.weight[FLOAT, 26x416x1x1]
  %blocks.3.1.se.conv_reduce.bias[FLOAT, 26]
  %blocks.3.1.se.conv_expand.weight[FLOAT, 416x26x1x1]
  %blocks.3.1.se.conv_expand.bias[FLOAT, 416]
  %blocks.3.2.se.conv_reduce.weight[FLOAT, 26x416x1x1]
  %blocks.3.2.se.conv_reduce.bias[FLOAT, 26]
  %blocks.3.2.se.conv_expand.weight[FLOAT, 416x26x1x1]
  %blocks.3.2.se.conv_expand.bias[FLOAT, 416]
  %blocks.3.3.se.conv_reduce.weight[FLOAT, 26x416x1x1]
  %blocks.3.3.se.conv_reduce.bias[FLOAT, 26]
  %blocks.3.3.se.conv_expand.weight[FLOAT, 416x26x1x1]
  %blocks.3.3.se.conv_expand.bias[FLOAT, 416]
  %blocks.4.0.se.conv_reduce.weight[FLOAT, 26x624x1x1]
  %blocks.4.0.se.conv_reduce.bias[FLOAT, 26]
  %blocks.4.0.se.conv_expand.weigh